### Comments from the TA
The proposal aims to predict a Kickstarter project's success/failure based on the title, category, and funding goals. The proposal is well written and interesting. 

Based on my previous experience, when you will train on the 2016 and 2017 projects and test in 2018, your evaluation will give you a low performance compared to the scenario where all the projects are mixed. 
* _Looks like the idea of cross validation or random sampling might work better?_

Intuitively, the kickstart's success depends on various factors, including how many video explanations are there, how descriptive the project description is, celebrity endorsements, and whether they have a prototype available. If you can get hands-on with such data, then I guess your predictive model will become stronger than just relying on title/category.
* _This would definitely be cool to have, but what might be a good way to get this? Looking it up manunally would obviously take too long. Maybe we can find another data set on kickstarter projects that have these other things?_
* _Here's the URL for one that has an extra field for "staff pick":_
    * https://www.kaggle.com/toshimelonhead/400000-kickstarter-projects


### Data Files

* 2018.csv
    * Original data that has been cleaned (removed NA values)
    
* 2018_5050_split.csv
    * Half are "success" and half are "failed"
    * Original data minus failed records
  
* 2018_reduced.csv
    * A sample of the 2018.csv
    * Size of sample = 20000

* 2018_5050_split_reduced.csv
    * Sample of 2018_5050_split.csv
    * Size of sample = 20000

### Questions + Thoughts for Later
* Do we only want to look at Successful vs Failed?
* Remove ID as well

### Data Characteristics
* 

### Potential Models
* Naive Bayes + Bag of Words (like in P1-3)
    * Will 
* Decision Trees
    * Not good with interacting attributes (are there any in this dataset?)
    * Data Explatory 
* We could try running kNN just to see how it works




* ANN - Quinn 
* Nam - Feature 
* Maria - Naive Bayes, Decision Trees, KNN
    * Stitch 2 datasets together
* Shuai - Keep working on data exploration
* Matt - Readibilty Feature 

In [1]:
import numpy as np
import pandas as pd

In [8]:
df_2018_ = pd.read_csv("../Data/Cleaned/2018.csv")
df_2018.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [9]:
df_2018.shape

(378661, 15)

In [10]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [11]:
df_2018.describe()

,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


In [12]:
df_2018["state"].value_counts(normalize=True)

failed        0.522153
successful    0.353762
canceled      0.102411
undefined     0.009407
live          0.007392
suspended     0.004875
Name: state, dtype: float64

In [17]:
df_2018.groupby(['main_category']).count()

,ID,name,category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
main_category,,,,,,,,,,,,,,
Art,28153,28152,28153,28153,28153,28153,28153,28153,28153,28153,28153,28153,28153,28153
Comics,10819,10819,10819,10819,10819,10819,10819,10819,10819,10819,10819,10819,10819,10819
Crafts,8809,8809,8809,8809,8809,8809,8809,8809,8809,8809,8809,8809,8809,8809
Dance,3768,3768,3768,3768,3768,3768,3768,3768,3768,3768,3768,3767,3768,3768
Design,30070,30069,30070,30070,30070,30070,30070,30070,30070,30070,30070,30067,30070,30070
Fashion,22816,22816,22816,22816,22816,22816,22816,22816,22816,22816,22816,22812,22816,22816
Film & Video,63585,63584,63585,63585,63585,63585,63585,63585,63585,63585,63585,62697,63585,63585
Food,24602,24602,24602,24602,24602,24602,24602,24602,24602,24602,24602,24599,24602,24602
Games,35231,35230,35231,35231,35231,35231,35231,35231,35231,35231,35231,35226,35231,35231


### Bag of Words (Taken from P1-3)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

In [26]:
name_state = df_2018[['name','state']].copy()
names = df_2018[['name']].copy()
names.head()

,name
0,The Songs of Adelaide & Abullah
1,Greeting From Earth: ZGAC Arts Capsule For ET
2,Where is Hank?
3,ToshiCapital Rekordz Needs Help to Complete Album
4,Community Film Project: The Art of Neighborhoo...


In [25]:
count_vector.fit(name)

doc_array = count_vector.transform(names).toarray()
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names())
frequency_matrix

,name,state
0,1,0


In [22]:
name_data = count_vector.fit_transform(name_state)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
# Need to decide the training set vs training 
#naive_bayes.fit(training_data,y_train)

In [ ]:
#Make prediction on test dataset
predictions = naive_bayes.predict(testing_data)